In [1]:
import numpy as np
import skfuzzy as fuzz

# Fungsi Keanggotaan Permintaan
def permintaan(x):
    turun = fuzz.trimf(x, [0, 0, 3000])
    tetap = fuzz.trimf(x, [2000, 3000, 4000])
    naik = fuzz.trimf(x, [3000, 6000, 6000])
    return turun, tetap, naik

# Fungsi Keanggotaan Persediaan
def persediaan(x):
    sedikit = fuzz.trimf(x, [0, 0, 500])
    sedang = fuzz.trimf(x, [200, 500, 800])
    banyak = fuzz.trimf(x, [500, 1000, 1000])
    return sedikit, sedang, banyak

# Fungsi Keanggotaan Produksi
def produksi(x):
    berkurang = fuzz.trimf(x, [0, 0, 5000])
    bertambah = fuzz.trimf(x, [2000, 10000, 10000])
    return berkurang, bertambah

# Inferensi Tsukamoto
def inferensi_tsukamoto(permintaan_input, persediaan_input):
    # Domain variabel
    x_permintaan = np.arange(0, 7001, 1)
    x_persediaan = np.arange(0, 1001, 1)
    x_produksi = np.arange(0, 10001, 1)

    # Keanggotaan fuzzy
    turun, tetap, naik = permintaan(x_permintaan)
    sedikit, sedang, banyak = persediaan(x_persediaan)
    berkurang, bertambah = produksi(x_produksi)

    # Derajat keanggotaan input
    permintaan_turun = fuzz.interp_membership(x_permintaan, turun, permintaan_input)
    permintaan_tetap = fuzz.interp_membership(x_permintaan, tetap, permintaan_input)
    permintaan_naik = fuzz.interp_membership(x_permintaan, naik, permintaan_input)

    persediaan_sedikit = fuzz.interp_membership(x_persediaan, sedikit, persediaan_input)
    persediaan_sedang = fuzz.interp_membership(x_persediaan, sedang, persediaan_input)
    persediaan_banyak = fuzz.interp_membership(x_persediaan, banyak, persediaan_input)

    # Inferensi aturan
    rules = []
    rules.append((min(permintaan_turun, persediaan_sedikit), bertambah))
    rules.append((min(permintaan_turun, persediaan_sedang), berkurang))
    rules.append((min(permintaan_turun, persediaan_banyak), berkurang))
    rules.append((min(permintaan_tetap, persediaan_sedikit), bertambah))
    rules.append((min(permintaan_tetap, persediaan_sedang), berkurang))
    rules.append((min(permintaan_tetap, persediaan_banyak), berkurang))
    rules.append((min(permintaan_naik, persediaan_sedikit), bertambah))
    rules.append((min(permintaan_naik, persediaan_sedang), bertambah))
    rules.append((min(permintaan_naik, persediaan_banyak), berkurang))

    # Perhitungan output
    produk = []
    for rule_strength, output in rules:
        z = fuzz.defuzz(x_produksi, output, 'centroid')
        produk.append(rule_strength * z)

    hasil = sum(produk) / sum(rule[0] for rule in rules)
    return hasil

# Contoh Penghitungan
permintaan_input = 2500  # Contoh input permintaan
persediaan_input = 300   # Contoh input persediaan
produksi_output = inferensi_tsukamoto(permintaan_input, persediaan_input)

print("Output produksi (unit):", produksi_output)


Output produksi (unit): 4677.08333333331
